In [1]:
import pathlib
import matplotlib.pyplot as plt
import utils
from torch import nn
from dataloaders import load_cifar10
from trainer import Trainer, compute_loss_and_accuracy
import os

In [39]:
class Model(nn.Module):

    def __init__(self,
                 image_channels,
                 num_classes):
        """
            Is called when model is initialized.
            Args:
                image_channels. Number of color channels in image (3)
                num_classes: Number of classes we want to predict (10)
        """
        super().__init__()
        # TODO: Implement this function (Task  2a)
        num_filters = [64, 128, 256]  # Set number of filters in each conv layer
        self.num_classes = num_classes
        # Define the first convolutional layer
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(
                in_channels=image_channels,
                out_channels=num_filters[0],
                kernel_size=5,
                stride=1,
                padding=2
            ),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = num_filters[0]),
            nn.MaxPool2d(
                kernel_size=2,
                stride=2
            )
        )
        # Define the rest of the convolutional layers
        for i in range(1, len(num_filters)):
            self.feature_extractor = nn.Sequential(
                self.feature_extractor,
                nn.Conv2d(
                    in_channels=num_filters[i-1],
                    out_channels=num_filters[i],
                    kernel_size=5,
                    stride=1,
                    padding=2
                ),
                nn.ReLU(),
                nn.BatchNorm2d(num_features = num_filters[i]),
                nn.MaxPool2d(
                    kernel_size=2,
                    stride=2
                )
            )

        self.num_output_features = 4*4*256
        # Initialize the first fully connected layer
        # Inputs all extracted features from the convolutional layers
        num_nodes = 64
        # Outputs num_nodes features
        self.classifier = nn.Sequential(
            nn.Linear(self.num_output_features, num_nodes),
            nn.ReLU(),
            nn.BatchNorm1d(num_features = num_nodes)
        )
        # Initialize the final fully connected layer
        # Inputs num_nodes features
        # Outputs num_classes predictions, 1 for each class.
        # There is no need for softmax activation function, as this is
        # included with nn.CrossEntropyLoss
        self.classifier = nn.Sequential(
            self.classifier,
            nn.Linear(num_nodes, num_classes),
        )

    def forward(self, x):
        """
        Performs a forward pass through the model
        Args:
            x: Input image, shape: [batch_size, 3, 32, 32]
        """
        # TODO: Implement this function (Task  2a)
        batch_size = x.shape[0]
        out = self.feature_extractor(x)
        out = out.view(batch_size, -1)
        out = self.classifier(out)
        expected_shape = (batch_size, self.num_classes)
        assert out.shape == (batch_size, self.num_classes),\
            f"Expected output of forward pass to be: {expected_shape}, but got: {out.shape}"
        return out

In [63]:
epochs = 10
batch_size = 32
learning_rate = 5e-2
early_stop_count = 4
dataloaders = load_cifar10(batch_size)
model = Model(image_channels=3, num_classes=10)
trainer = Trainer(
    batch_size,
    learning_rate,
    early_stop_count,
    epochs,
    model,
    dataloaders
)
trainer.train()

Files already downloaded and verified
Files already downloaded and verified
Model(
  (feature_extractor): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (1): ReLU()
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (2): ReLU()
      (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): ReLU()
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )


In [64]:
trainer.load_best_model()

dataloader_train, dataloader_val, dataloader_test = dataloaders
train_loss, train_acc = compute_loss_and_accuracy(
    dataloader_train, model, nn.CrossEntropyLoss()
)
validation_loss, validation_acc = compute_loss_and_accuracy(
    dataloader_val, model, nn.CrossEntropyLoss()
)
test_loss, test_acc = compute_loss_and_accuracy(
    dataloader_test, model, nn.CrossEntropyLoss()
)
print("Best model training accuracy:", train_acc)
print("Best model validation accuracy:", validation_acc)
print("Best model test accuracy:", test_acc)

Best model training accuracy: 0.8979596372688478
Best model validation accuracy: 0.7664
Best model test accuracy: 0.7546
